In [ ]:
# Install Chrome, ChromeDriver, and Selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup headless Chrome for Colab
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

# Load the page
url = "https://www.counterpointresearch.com/insights/us-smartphone-market-share/"
driver.get(url)
time.sleep(5)  # give time for JavaScript to load

# Parse the rendered HTML
soup = BeautifulSoup(driver.page_source, 'html.parser')
tables = soup.find_all('table')

apple_samsung_tables = []

# Search for relevant data
for table in tables:
    rows = table.find_all('tr')
    headers = [th.text.strip() for th in rows[0].find_all(['th', 'td'])]

    if 'Brands' not in headers[0] or len(headers) < 3:
        continue

    data = []
    for row in rows[1:]:
        cells = [td.text.strip() for td in row.find_all('td')]
        if not cells:
            continue
        if cells[0] in ['Apple', 'Samsung']:
            data.append([cells[0]] + [cell.replace('%','') for cell in cells[1:]])


    if data:
        df = pd.DataFrame(data, columns=headers)
        df.set_index('Brands', inplace=True)
        df = df.transpose()
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Quarter'}, inplace=True)
        apple_samsung_tables.append(df)

driver.quit()

# Combine and show final data
final_df = pd.concat(apple_samsung_tables).drop_duplicates(subset='Quarter').reset_index(drop=True)
final_df.columns.name = None
final_df[['Apple', 'Samsung']] = final_df[['Apple', 'Samsung']].apply(pd.to_numeric)
print("Apple and Samsung Market Share Data:")
print(final_df)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading